In [1]:
!pip install -q pandas numpy seaborn matplotlib mlxtend pyfim kagglehub


# Mineria de patrones - Online Retail

Frequent itemsets, reglas de asociacion y patrones emergentes sobre el dataset limpio. Incluye opcion de preprocesamiento en GPU (cudf) y conversion a pandas para FP-Growth.


## Objetivos
- Preparar transacciones con un unico espacio de items (encoder unico)
- Obtener itemsets frecuentes y reglas (soporte, confianza, lift)
- Comparar particiones temporales (H1 vs H2 2011) con growth rate
- Exportar resultados y graficas para observabilidad


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from mlxtend.frequent_patterns import fpgrowth, association_rules, fpmax
from mlxtend.preprocessing import TransactionEncoder

# GPU opcional
try:
    import cudf
    USE_GPU_DEFAULT = True
except ImportError:
    cudf = None
    USE_GPU_DEFAULT = False

sns.set_theme(style="whitegrid")
pd.options.display.float_format = "{:,.4f}".format


## Configuracion


In [3]:
DATA_DIR = Path("data")
PROCESSED_PATH = DATA_DIR / "processed" / "cleaned_online_retail.csv"
RESULTS_DIR = DATA_DIR / "processed"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Parametros
MIN_SUPPORT = 0.01
MAX_LEN = None  # ejemplo: 3 para limitar
MIN_CONFIDENCE = 0.3
SEED = 42
np.random.seed(SEED)
USE_GPU = USE_GPU_DEFAULT  # cambia a False si quieres forzar CPU
print(f"USE_GPU={USE_GPU} (cudf {'disponible' if cudf else 'no disponible'})")


USE_GPU=False (cudf no disponible)


## Carga del dataset procesado


In [4]:
if not PROCESSED_PATH.exists():
    raise FileNotFoundError("Ejecuta primero eda.ipynb para generar cleaned_online_retail.csv")

df_pd = pd.read_csv(PROCESSED_PATH)
df_pd['InvoiceDate'] = pd.to_datetime(df_pd['InvoiceDate'])

df = cudf.from_pandas(df_pd) if (USE_GPU and cudf is not None) else df_pd
print(f"Filas: {len(df_pd):,} | Items unicos: {df_pd['Description'].nunique()} | Facturas: {df_pd['InvoiceNo'].nunique()}")
print(df_pd.head())
print(f"DataFrame en {'GPU' if USE_GPU and cudf else 'CPU'}")


Filas: 494,588 | Items unicos: 2190 | Facturas: 19748
   InvoiceNo StockCode                          Description  Quantity  \
0     536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER    6.0000   
1     536365     71053                  WHITE METAL LANTERN    6.0000   
2     536365    84406B       CREAM CUPID HEARTS COAT HANGER    8.0000   
3     536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE    6.0000   
4     536365    84029E       RED WOOLLY HOTTIE WHITE HEART.    6.0000   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00     2.5500 17,850.0000  United Kingdom  
1 2010-12-01 08:26:00     3.3900 17,850.0000  United Kingdom  
2 2010-12-01 08:26:00     2.7500 17,850.0000  United Kingdom  
3 2010-12-01 08:26:00     3.3900 17,850.0000  United Kingdom  
4 2010-12-01 08:26:00     3.3900 17,850.0000  United Kingdom  
DataFrame en CPU


## Particiones temporales
Se separa 2011 en dos mitades para observar cambios estacionales.


In [5]:
cutoff = pd.Timestamp('2011-06-01')
part1 = df[df['InvoiceDate'] < cutoff]
part2 = df[df['InvoiceDate'] >= cutoff]

print(f"H1: {len(part1):,} filas, {part1['InvoiceNo'].nunique()} facturas")
print(f"H2: {len(part2):,} filas, {part2['InvoiceNo'].nunique()} facturas")


H1: 188,150 filas, 8024 facturas
H2: 306,438 filas, 11724 facturas


## Preparacion de transacciones (encoder unico)
Se ajusta `TransactionEncoder` con todas las transacciones para garantizar el mismo espacio de items en cada particion. Se usa pandas para el encoding, pero los filtros previos pueden correr en GPU.


In [6]:
def to_pandas(df_like):
    return df_like.to_pandas() if hasattr(df_like, 'to_pandas') else df_like


def build_transactions(df_part_pd):
    return df_part_pd.groupby('InvoiceNo')['Description'].apply(list).tolist()

# Convertimos a pandas para TransactionEncoder
part1_pd = to_pandas(part1)
part2_pd = to_pandas(part2)
df_all_pd = to_pandas(df)

transactions_all = build_transactions(df_all_pd)
transactions1 = build_transactions(part1_pd)
transactions2 = build_transactions(part2_pd)

te = TransactionEncoder()
te_ary_all = te.fit(transactions_all).transform(transactions_all)
onehot_all = pd.DataFrame(te_ary_all, columns=te.columns_)

# Reusar las mismas columnas para las particiones
te_ary_1 = te.transform(transactions1)
onehot_1 = pd.DataFrame(te_ary_1, columns=te.columns_)

te_ary_2 = te.transform(transactions2)
onehot_2 = pd.DataFrame(te_ary_2, columns=te.columns_)

print(f"Items en espacio comun: {onehot_all.shape[1]}")
print(f"Transacciones totales: {len(transactions_all):,} | H1: {len(transactions1):,} | H2: {len(transactions2):,}")
print(f"Densidad promedio one-hot (ALL): {onehot_all.values.mean():.4f}")


Items en espacio comun: 2190
Transacciones totales: 19,748 | H1: 8,024 | H2: 11,724
Densidad promedio one-hot (ALL): 0.0113


## Funcion auxiliar para itemsets


In [7]:
def run_fpgrowth(df_oh, min_support=MIN_SUPPORT, max_len=MAX_LEN):
    freq = fpgrowth(df_oh, min_support=min_support, use_colnames=True, max_len=max_len)
    freq = freq.sort_values(by='support', ascending=False).reset_index(drop=True)
    return freq


def top_summary(freq, name, n=5):
    print(f"Top {n} itemsets frecuentes - {name}")
    print(freq.head(n))

freq_all = run_fpgrowth(onehot_all)
freq_p1 = run_fpgrowth(onehot_1)
freq_p2 = run_fpgrowth(onehot_2)

top_summary(freq_all, "Dataset completo")
top_summary(freq_p1, "H1")
top_summary(freq_p2, "H2")


KeyboardInterrupt: 

## Reglas de asociacion (dataset completo)


In [ ]:
rules = association_rules(freq_all, metric='confidence', min_threshold=MIN_CONFIDENCE)
rules = rules.sort_values(by='lift', ascending=False)
print(f"Reglas generadas: {len(rules)}")
print(rules[['antecedents','consequents','support','confidence','lift']].head())

plt.figure(figsize=(8,5))
plt.scatter(rules['support'], rules['confidence'], c=rules['lift'], cmap='viridis', alpha=0.6)
plt.colorbar(label='Lift')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Mapa Support vs Confidence (color = Lift)')
plt.tight_layout()
plt.show()


## Patrones emergentes (growth rate H2 vs H1)
Se calcula growth rate comparando soportes de los mismos itemsets entre H1 y H2. Los inf son Jumping Emerging Patterns (JEP).


In [ ]:
merged = pd.merge(freq_p1, freq_p2, on='itemsets', how='outer', suffixes=('_1', '_2')).fillna(0)
merged['growth_rate'] = np.where(
    merged['support_1'] == 0,
    np.inf,
    merged['support_2'] / merged['support_1']
)

merged = merged.sort_values(by='growth_rate', ascending=False)

# Segmentos
jep = merged[merged['growth_rate'] == np.inf].sort_values(by='support_2', ascending=False)
inc = merged[(merged['growth_rate'] > 1) & (merged['growth_rate'] != np.inf)].sort_values(by='growth_rate', ascending=False)
stable = merged[(merged['growth_rate'] >= 0.8) & (merged['growth_rate'] <= 1.2)].sort_values(by='support_2', ascending=False)
dec = merged[(merged['growth_rate'] < 0.8) & (merged['support_2'] > 0)].sort_values(by='growth_rate')

print(f"Itemsets comparados: {len(merged)}")
print(f"JEPs: {len(jep)} | Incrementan: {len(inc)} | Estables: {len(stable)} | Decrecen: {len(dec)}")
print("Top 5 JEP (por soporte H2):")
print(jep.head())


## Visualizaciones de patrones


In [ ]:
def iset_to_str(s):
    try:
        return ', '.join(sorted(s))
    except Exception:
        return str(s)

# Top growth (sin inf)
top_inc = inc.head(10).copy()
top_inc['item'] = top_inc['itemsets'].apply(iset_to_str)

plt.figure(figsize=(10,4))
plt.bar(top_inc['item'], top_inc['growth_rate'], color='teal')
plt.xticks(rotation=60, ha='right')
plt.ylabel('Growth rate (H2/H1)')
plt.title('Top 10 Emerging Patterns (sin JEP)')
plt.tight_layout()
plt.show()

# JEPs mas soportados
top_jep = jep.head(10).copy()
top_jep['item'] = top_jep['itemsets'].apply(iset_to_str)
plt.figure(figsize=(8,4))
sns.barplot(x=top_jep['support_2'], y=top_jep['item'], palette='flare')
plt.xlabel('Support H2')
plt.title('Top 10 Jumping Emerging Patterns')
plt.tight_layout()
plt.show()

# Top soportes por particion
for name, freq_df in [("H1", freq_p1), ("H2", freq_p2)]:
    top_support = freq_df.head(10).copy()
    top_support['item'] = top_support['itemsets'].apply(iset_to_str)
    plt.figure(figsize=(8,4))
    sns.barplot(x=top_support['support'], y=top_support['item'], palette='crest')
    plt.title(f'Top 10 itemsets por soporte - {name}')
    plt.xlabel('Support')
    plt.tight_layout()
    plt.show()


## Exportar resultados


In [ ]:
freq_all.to_csv(RESULTS_DIR / 'frequent_itemsets_all.csv', index=False)
freq_p1.to_csv(RESULTS_DIR / 'frequent_itemsets_h1.csv', index=False)
freq_p2.to_csv(RESULTS_DIR / 'frequent_itemsets_h2.csv', index=False)
rules.to_csv(RESULTS_DIR / 'association_rules_all.csv', index=False)
merged.to_csv(RESULTS_DIR / 'growth_rates_h1_vs_h2.csv', index=False)
print("Archivos guardados en data/processed/")
